In [13]:
pip install "numpy<2.0"  


   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
   --- ------------------------------------ 1.3/15.5 MB 7.4 MB/s eta 0:00:02
   -------- ------------------------------- 3.4/15.5 MB 8.8 MB/s eta 0:00:02
   -------------------- ------------------- 8.1/15.5 MB 13.6 MB/s eta 0:00:01
   ---------------------------------------  15.2/15.5 MB 19.1 MB/s eta 0:00:01
   ---------------------------------------- 15.5/15.5 MB 18.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
pip install tqdm

In [ ]:
pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 298.3 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=8e8c32ed191bf5767f078bc91b41c063c5770561193dc26c20c2f5bccda34f83
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
pip install pytorch-crf


In [32]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from functools import partial
import uuid
from collections import Counter
from seqeval.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np
import logging
import re


In [2]:
import torch
import torch.nn as nn
from torchcrf import CRF
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from collections import Counter


In [50]:
import logging

In [49]:
import re

In [5]:
class BiLSTMCRFForTokenClassification(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_labels, dropout=0.1):
        super(BiLSTMCRFForTokenClassification, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim * 2, num_labels)
        self.crf = CRF(num_labels, batch_first=True)

    def forward(self, input_ids, attention_mask=None, labels=None):
        embedded = self.embedding(input_ids)
        lstm_out, _ = self.lstm(embedded)
        lstm_out = self.dropout(lstm_out)
        emissions = self.fc(lstm_out)
        
        if labels is not None:
            mask = attention_mask.bool()
            loss = -self.crf(emissions, labels, mask=mask)
            return loss
        else:
            mask = attention_mask.bool()
            predictions = self.crf.decode(emissions, mask=mask)
            return predictions

In [6]:
class PIIDataset(Dataset):
    def __init__(self, examples, max_length):
        self.examples = examples
        self.max_length = max_length

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        item = self.examples[idx]
        input_ids = torch.tensor(item['input_ids'], dtype=torch.long)
        labels = torch.tensor(item['labels'], dtype=torch.long)
        attention_mask = torch.ones_like(input_ids)
        
        input_ids = input_ids[:self.max_length]
        labels = labels[:self.max_length]
        attention_mask = attention_mask[:self.max_length]
        
        input_ids = torch.nn.functional.pad(input_ids, (0, self.max_length - len(input_ids)), value=0)
        labels = torch.nn.functional.pad(labels, (0, self.max_length - len(labels)), value=-100)
        attention_mask = torch.nn.functional.pad(attention_mask, (0, self.max_length - len(attention_mask)), value=0)
        
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }

In [7]:
def process_data(example, labels2int, vocab, max_length):
    tokens = example["source_text"].split()
    labels = ["O"] * len(tokens)

    for label in example["privacy_mask"]:
        start_char, end_char = label['start'], label['end']
        start_token = len(example["source_text"][:start_char].split())
        end_token = min(len(example["source_text"][:end_char].split()), len(tokens))

        label_type = label['label']
        if label_type != "CARDISSUER":  # Skip CARDISSUER labels
            if label_type.startswith("GIVENNAME"):
                label_type = "GIVENNAME"
            elif label_type.startswith("LASTNAME"):
                label_type = "LASTNAME"

            if start_token < len(labels):
                labels[start_token] = f'B-{label_type}'
                for i in range(start_token + 1, end_token):
                    if i < len(labels):
                        labels[i] = f'I-{label_type}'

    input_ids = [vocab.get(token, vocab['<UNK>']) for token in tokens[:max_length]]
    label_ids = [labels2int[label] for label in labels[:max_length]]

    input_ids = input_ids[:max_length] + [vocab['<PAD>']] * (max_length - len(input_ids))
    label_ids = label_ids[:max_length] + [labels2int['O']] * (max_length - len(label_ids))

    return {
        'input_ids': input_ids,
        'labels': label_ids
    }

In [8]:
def train(model, train_loader, val_loader, optimizer, device, num_epochs, id2label, patience=3):
    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_model = None
    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            loss = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            progress_bar.set_postfix({'train_loss': f"{loss.item():.4f}"})

        avg_train_loss = total_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        print(f"\nEpoch {epoch+1}/{num_epochs}, Average training loss: {avg_train_loss:.4f}")

        model.eval()
        val_loss = 0
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validation"):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                loss = model(input_ids, attention_mask=attention_mask, labels=labels)
                val_loss += loss.item()

                preds = model(input_ids, attention_mask=attention_mask)

                for i, mask in enumerate(attention_mask):
                    pred = [id2label[p] for p, m in zip(preds[i], mask) if m.item() == 1]
                    true = [id2label[l.item()] for l, m in zip(labels[i], mask) if m.item() == 1]
                    if pred and true:
                        all_preds.append(pred)
                        all_labels.append(true)

        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        print(f"Validation loss: {avg_val_loss:.4f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
            best_model = model.state_dict()
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                print(f"Early stopping triggered after epoch {epoch+1}")
                model.load_state_dict(best_model)
                break

        print("Label distribution in true labels:")
        print(Counter(sum(all_labels, [])))

        print("\nLabel distribution in predictions:")
        print(Counter(sum(all_preds, [])))

        print(f"\nNumber of sequences evaluated: {len(all_labels)}")
        print(f"Number of sequences with predictions: {len(all_preds)}")
        print(f"Number of sequences with labels: {len(all_labels)}")

        print("\nClassification Report:")
        print(classification_report(all_labels, all_preds))

    # Plot learning curve
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
    plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Learning Curve')
    plt.legend()
    plt.savefig('learning_curve.png')
    plt.close()

    return model, all_labels, all_preds

In [9]:
def plot_confusion_matrix(true_labels, pred_labels, label_list):
    true_flat = [item for sublist in true_labels for item in sublist]
    pred_flat = [item for sublist in pred_labels for item in sublist]

    true_counts = Counter(true_flat)
    pred_counts = Counter(pred_flat)
    print("True label counts:", true_counts)
    print("Pred label counts:", pred_counts)

    cm = confusion_matrix(true_flat, pred_flat, labels=label_list)

    plt.figure(figsize=(20, 16))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_list, yticklabels=label_list)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig('confusion_matrix.png')
    plt.close()

    print(f"Confusion matrix saved as 'confusion_matrix.png'")


In [14]:
def main():
    labels = [
        "BOD", "BUILDING", "CITY", "COUNTRY", "DATE", "DRIVERLICENSE",
        "EMAIL", "GEOCOORD", "GIVENNAME", "IDCARD", "IP", "LASTNAME",
        "PASS", "PASSPORT", "POSTCODE", "SECADDRESS", "SEX",
        "SOCIALNUMBER", "STATE", "STREET", "TEL", "TIME", "TITLE", "USERNAME"
    ]

    label_list = ["O"] + [f"B-{label}" for label in labels] + [f"I-{label}" for label in labels]
    label2id = {label: i for i, label in enumerate(label_list)}
    id2label = {v: k for k, v in label2id.items()}

    ds = load_dataset("ai4privacy/pii-masking-300k")
    ds = ds.filter(lambda x: x["language"] == "English", num_proc=4)
    ds = ds.filter(lambda x: "CARDISSUER" not in x["mbert_bio_labels"], num_proc=4)

    vocab = {'<PAD>': 0, '<UNK>': 1}
    for example in ds['train']:
        for token in example['source_text'].split():
            if token not in vocab:
                vocab[token] = len(vocab)

    max_length = 512
    ds = ds.map(
        partial(process_data, labels2int=label2id, vocab=vocab, max_length=max_length),
        remove_columns=['privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set', 'source_text'],
        num_proc=8
    )

    val_test_split = 0.3
    val_dataset = ds["validation"]
    val_dataset = val_dataset.shuffle(seed=42)
    val_size = int(len(val_dataset) * (1 - val_test_split))
    test_size = len(val_dataset) - val_size
    val_dataset, test_dataset = val_dataset.select(range(val_size)), val_dataset.select(range(val_size, len(val_dataset)))

    train_dataset = PIIDataset(ds["train"], max_length)
    val_dataset = PIIDataset(val_dataset, max_length)
    test_dataset = PIIDataset(test_dataset, max_length)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64)
    test_loader = DataLoader(test_dataset, batch_size=64)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    model = BiLSTMCRFForTokenClassification(
        vocab_size=len(vocab),
        embedding_dim=100,
        hidden_dim=128,
        num_labels=len(label_list)
    ).to(device)
    print(model)

    optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)

    num_epochs = 3
    print(f"Starting training for {num_epochs} epochs...")
    model, true_labels, pred_labels = train(model, train_loader, val_loader, optimizer, device, num_epochs, id2label)

    model_save_path = f"bilstm_crf_pii_model_{str(uuid.uuid4())}.pth"
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved successfully to {model_save_path}")

    model.eval()
    test_preds = []
    test_labels = []
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Testing"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            preds = model(input_ids, attention_mask=attention_mask)

            for i, mask in enumerate(attention_mask):
                pred = [id2label[p] for p, m in zip(preds[i], mask) if m.item() == 1]
                true = [id2label[l.item()] for l, m in zip(labels[i], mask) if m.item() == 1]
                if pred and true:
                    test_preds.append(pred)
                    test_labels.append(true)

    print("\nTest Results:")
    print(classification_report(test_labels, test_preds))

    plot_confusion_matrix(test_labels, test_preds, label_list)

In [15]:
if __name__ == "__main__":
    main()

Using device: cuda
BiLSTMCRFForTokenClassification(
  (embedding): Embedding(232271, 100)
  (lstm): LSTM(100, 128, batch_first=True, bidirectional=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (fc): Linear(in_features=256, out_features=49, bias=True)
  (crf): CRF(num_tags=49)
)
Starting training for 3 epochs...


Epoch 1/3: 100%|████████████████████████████████████████████████| 935/935 [06:11<00:00,  2.52it/s, train_loss=410.3184]



Epoch 1/3, Average training loss: 1284.7368


Validation: 100%|██████████████████████████████████████████████████████████████████████| 87/87 [02:18<00:00,  1.60s/it]


Validation loss: 1392.5552
Label distribution in true labels:
Counter({'O': 2804958, 'B-TIME': 2578, 'B-LASTNAME': 2166, 'B-USERNAME': 1960, 'B-IDCARD': 1785, 'B-EMAIL': 1779, 'B-SOCIALNUMBER': 1711, 'B-DRIVERLICENSE': 1680, 'B-PASSPORT': 1673, 'B-GIVENNAME': 1672, 'B-BOD': 1644, 'B-IP': 1511, 'I-STREET': 1410, 'B-CITY': 1384, 'B-SEX': 1382, 'B-STATE': 1363, 'B-TEL': 1362, 'B-STREET': 1327, 'B-TITLE': 1321, 'B-POSTCODE': 1305, 'B-BUILDING': 1279, 'B-DATE': 1192, 'B-PASS': 1162, 'B-COUNTRY': 1105, 'I-BOD': 881, 'I-TEL': 782, 'I-TIME': 735, 'I-DATE': 684, 'B-SECADDRESS': 595, 'I-SOCIALNUMBER': 555, 'I-DRIVERLICENSE': 522, 'I-CITY': 500, 'I-SECADDRESS': 374, 'I-POSTCODE': 374, 'I-COUNTRY': 297, 'I-SEX': 191, 'B-GEOCOORD': 152, 'I-GEOCOORD': 96, 'I-PASS': 74, 'I-USERNAME': 69, 'I-LASTNAME': 60, 'I-EMAIL': 44, 'I-TITLE': 16, 'I-STATE': 15, 'I-GIVENNAME': 8, 'I-IDCARD': 6, 'I-PASSPORT': 5})

Label distribution in predictions:
Counter({'O': 2817575, 'B-TIME': 3604, 'B-STATE': 1862, 'B-LASTNAM

...


Epoch 3/3, Average training loss: 391.3133


Validation: 100%|██████████████████████████████████████████████████████████████████████| 87/87 [02:16<00:00,  1.56s/it]


Validation loss: 993.0304
Label distribution in true labels:
Counter({'O': 2804958, 'B-TIME': 2578, 'B-LASTNAME': 2166, 'B-USERNAME': 1960, 'B-IDCARD': 1785, 'B-EMAIL': 1779, 'B-SOCIALNUMBER': 1711, 'B-DRIVERLICENSE': 1680, 'B-PASSPORT': 1673, 'B-GIVENNAME': 1672, 'B-BOD': 1644, 'B-IP': 1511, 'I-STREET': 1410, 'B-CITY': 1384, 'B-SEX': 1382, 'B-STATE': 1363, 'B-TEL': 1362, 'B-STREET': 1327, 'B-TITLE': 1321, 'B-POSTCODE': 1305, 'B-BUILDING': 1279, 'B-DATE': 1192, 'B-PASS': 1162, 'B-COUNTRY': 1105, 'I-BOD': 881, 'I-TEL': 782, 'I-TIME': 735, 'I-DATE': 684, 'B-SECADDRESS': 595, 'I-SOCIALNUMBER': 555, 'I-DRIVERLICENSE': 522, 'I-CITY': 500, 'I-SECADDRESS': 374, 'I-POSTCODE': 374, 'I-COUNTRY': 297, 'I-SEX': 191, 'B-GEOCOORD': 152, 'I-GEOCOORD': 96, 'I-PASS': 74, 'I-USERNAME': 69, 'I-LASTNAME': 60, 'I-EMAIL': 44, 'I-TITLE': 16, 'I-STATE': 15, 'I-GIVENNAME': 8, 'I-IDCARD': 6, 'I-PASSPORT': 5})

Label distribution in predictions:
Counter({'O': 2805007, 'B-TIME': 4334, 'I-TEL': 3650, 'B-LASTNAME':

Testing: 100%|█████████████████████████████████████████████████████████████████████████| 38/38 [01:06<00:00,  1.75s/it]



Test Results:
               precision    recall  f1-score   support

          BOD       0.80      0.56      0.66       657
     BUILDING       0.77      0.72      0.74       547
         CITY       0.70      0.65      0.68       598
      COUNTRY       0.88      0.89      0.88       436
         DATE       0.52      0.61      0.56       480
DRIVERLICENSE       0.62      0.48      0.54       692
        EMAIL       0.70      0.55      0.62       739
     GEOCOORD       0.48      0.31      0.38        64
    GIVENNAME       0.54      0.27      0.36       651
       IDCARD       0.72      0.43      0.54       812
           IP       0.79      0.54      0.64       624
     LASTNAME       0.34      0.43      0.38       974
         PASS       0.84      0.52      0.64       435
     PASSPORT       0.76      0.53      0.62       718
     POSTCODE       0.51      0.72      0.60       570
   SECADDRESS       0.79      0.70      0.74       264
          SEX       0.92      0.74      0.82     